In [31]:
# Import libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import validation_curve


In [41]:
data_train= pd.read_csv('dataset/data_train.csv')
data_test = pd.read_csv('dataset/data_test.csv')
df_train=data_train.copy()
df_test=data_test.copy()
pd.set_option('display.max_columns',111)
print('train.shape=',df_train.shape)
print('test.shape=',df_test.shape)
df_train.head()
#print(df_test.head())

train.shape= (32561, 15)
test.shape= (16281, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,0,77516,0,13,0,0,0,0,0,2174,0,40,0,0
1,50,1,83311,0,13,1,1,1,0,0,0,0,13,0,0
2,38,2,215646,1,9,2,2,0,0,0,0,0,40,0,0
3,53,2,234721,2,7,1,2,1,1,0,0,0,40,0,0
4,28,2,338409,0,13,1,3,2,1,1,0,0,40,1,0


In [16]:
print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   age             32561 non-null  int64
 1   workclass       32561 non-null  int64
 2   fnlwgt          32561 non-null  int64
 3   education       32561 non-null  int64
 4   education-num   32561 non-null  int64
 5   marital-status  32561 non-null  int64
 6   occupation      32561 non-null  int64
 7   relationship    32561 non-null  int64
 8   race            32561 non-null  int64
 9   sex             32561 non-null  int64
 10  capital-gain    32561 non-null  int64
 11  capital-loss    32561 non-null  int64
 12  hours-per-week  32561 non-null  int64
 13  native-country  32561 non-null  int64
 14  income          32561 non-null  int64
dtypes: int64(15)
memory usage: 3.7 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 15 columns):
 #   Co

In [28]:
# Standardizing the dataset
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve

# define X and y for the models:
X_train=df_train.drop('income',axis=1)
y_train=df_train['income']
X_test=df_test.drop('income',axis=1)
y_test=df_test['income']
# X and y total:
X = pd.concat([X_train, X_test],axis=0)
y = pd.concat([y_train, y_test],axis=0)

X_train= preprocessing.StandardScaler().fit(X_train).transform(X_train)
X_test= preprocessing.StandardScaler().fit(X_test).transform(X_test)

#X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=0)
print ('Train set:', X_train.shape, y_train.shape)
print ('Test set :', X_test.shape, y_test.shape)
print ('Total set :', X.shape, y.shape)

Train set: (32561, 14) (32561,)
Test set : (16281, 14) (16281,)
Total set : (48842, 14) (48842,)


In [32]:
# Random Forest:

# Train and fit the Random Forest Classification model
forest = RandomForestClassifier(n_estimators=100,random_state = 0)

forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

print(confusion_matrix(y_test, y_pred))
# Classification report
print(metrics.classification_report(y_test, y_pred.round(), digits=3))
# Calculate cv score with 'roc_auc_ovr' scoring and 10 folds
accuracy = cross_val_score(forest, X, y,scoring = 'roc_auc_ovr',cv=10)
print('cross validation score with roc_auc_ovr scoring=',accuracy.mean())

# Calculate roc_auc score
print('roc_auc_score=',roc_auc_score(y_test,forest.predict_proba(X_test)[:, 1]))


[[11564   871]
 [ 1565  2281]]
              precision    recall  f1-score   support

           0      0.881     0.930     0.905     12435
           1      0.724     0.593     0.652      3846

    accuracy                          0.850     16281
   macro avg      0.802     0.762     0.778     16281
weighted avg      0.844     0.850     0.845     16281

cross validation score with roc_auc_ovr scoring= 0.9069010222454592
roc_auc_score= 0.9007813275940768


In [36]:
# Random Forest: with GridSearchCV to improve the score

# Train and fit the Random Forest Classification model
forest = RandomForestClassifier(n_estimators=100,random_state = 0)

# GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2']
}

# with Kfold=10
CV_rfc = GridSearchCV(estimator=forest, param_grid=param_grid, cv= 15)
CV_rfc.fit(X_train, y_train)
print('best_estimator = ',CV_rfc.best_estimator_)
y_pred = CV_rfc.predict(X_test)

print(confusion_matrix(y_test, y_pred))
# Classification report
print(metrics.classification_report(y_test, y_pred.round(), digits=3))
# Calculate cv score with 'roc_auc_ovr' scoring and 10 Kfolds
accuracy = cross_val_score(forest, X, y,scoring = 'roc_auc_ovr',cv=10)
print('cross validation score with roc_auc_ovr scoring=',accuracy.mean())

# Calculate roc_auc score
print('roc_auc_score=',roc_auc_score(y_test,CV_rfc.predict_proba(X_test)[:, 1]))

best_estimator =  RandomForestClassifier(n_estimators=300, random_state=0)
[[11547   888]
 [ 1529  2317]]
              precision    recall  f1-score   support

           0      0.883     0.929     0.905     12435
           1      0.723     0.602     0.657      3846

    accuracy                          0.852     16281
   macro avg      0.803     0.766     0.781     16281
weighted avg      0.845     0.852     0.847     16281

cross validation score with roc_auc_ovr scoring= 0.9069010222454592
roc_auc_score= 0.9027798007778776


In [37]:
# Random Forest: with GridSearchCV to improve the score

# Train and fit the Random Forest Classification model
forest = RandomForestClassifier(n_estimators=100,random_state = 0)

# GridSearchCV
param_grid = {
    'n_estimators': [350, 400, 450],
    'max_features': ['auto', 'sqrt', 'log2']
}

# with Kfold=20
CV_rfc = GridSearchCV(estimator=forest, param_grid=param_grid, cv= 20)
CV_rfc.fit(X_train, y_train)
print('best_estimator = ',CV_rfc.best_estimator_)
y_pred = CV_rfc.predict(X_test)

print(confusion_matrix(y_test, y_pred))
# Classification report
print(metrics.classification_report(y_test, y_pred.round(), digits=3))
# Calculate cv score with 'roc_auc_ovr' scoring and 20 Kfolds
accuracy = cross_val_score(forest, X, y,scoring = 'roc_auc_ovr',cv=20)
print('cross validation score with roc_auc_ovr scoring=',accuracy.mean())

# Calculate roc_auc score
print('roc_auc_score=',roc_auc_score(y_test,CV_rfc.predict_proba(X_test)[:, 1]))

best_estimator =  RandomForestClassifier(n_estimators=450, random_state=0)
[[11546   889]
 [ 1536  2310]]
              precision    recall  f1-score   support

           0      0.883     0.929     0.905     12435
           1      0.722     0.601     0.656      3846

    accuracy                          0.851     16281
   macro avg      0.802     0.765     0.780     16281
weighted avg      0.845     0.851     0.846     16281

cross validation score with roc_auc_ovr scoring= 0.9069010222454592
roc_auc_score= 0.9031634180526047


In [38]:
# Calculate cv score with 'roc_auc_ovr' scoring and 20 Kfolds
accuracy = cross_val_score(forest, X, y,scoring = 'roc_auc_ovr',cv=20)
print('cross validation score with roc_auc_ovr scoring=',accuracy.mean())

cross validation score with roc_auc_ovr scoring= 0.906701649426753


In [39]:
# Random Forest: with GridSearchCV to improve the score

# Train and fit the Random Forest Classification model
forest = RandomForestClassifier(n_estimators=100,random_state = 0)

# GridSearchCV
param_grid = {
    'n_estimators': [500, 550, 600],
    'max_features': ['auto', 'sqrt', 'log2']
}

# with Kfold=20
CV_rfc = GridSearchCV(estimator=forest, param_grid=param_grid, cv= 20)
CV_rfc.fit(X_train, y_train)
print('best_estimator = ',CV_rfc.best_estimator_)
y_pred = CV_rfc.predict(X_test)

print(confusion_matrix(y_test, y_pred))
# Classification report
print(metrics.classification_report(y_test, y_pred.round(), digits=3))
# Calculate cv score with 'roc_auc_ovr' scoring and 20 Kfolds
accuracy = cross_val_score(forest, X, y,scoring = 'roc_auc_ovr',cv=20)
print('cross validation score with roc_auc_ovr scoring=',accuracy.mean())

# Calculate roc_auc score
print('roc_auc_score=',roc_auc_score(y_test,CV_rfc.predict_proba(X_test)[:, 1]))

best_estimator =  RandomForestClassifier(n_estimators=600, random_state=0)
[[11558   877]
 [ 1531  2315]]
              precision    recall  f1-score   support

           0      0.883     0.929     0.906     12435
           1      0.725     0.602     0.658      3846

    accuracy                          0.852     16281
   macro avg      0.804     0.766     0.782     16281
weighted avg      0.846     0.852     0.847     16281

cross validation score with roc_auc_ovr scoring= 0.906701649426753
roc_auc_score= 0.9033553155556056
